In [2]:
from IPython.display import clear_output
import pandas as pd
import numpy as np
from calendar import month_abbr
from functools import reduce
from IPython.display import clear_output
import dateutil

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly, plot_cross_validation_metric
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.utilities import regressor_coefficients

import os
import json
from fbprophet.serialize import model_to_json, model_from_json

from scipy.stats import normaltest
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests
from statsmodels.api import OLS
from src.helpers import *


model_names = ['indices', 'tele_diff', 'lagged_tele', 'elnino_cat', 'lagged_elnino', 'ideal_lag', 'overfitted']
model_calls = [get_temp_and_indices, get_temp_diff_tele_indices, get_temp_lagged_tele_indices, get_temp_elnino_cat_indices, get_temp_lagged_elnino,
              get_temp_ideal_lag, get_temp_overfitted]
cities = ['Singapore', 'Tokyo', 'Toronto', 'Delhi', 'Cairo', 'Riyadh', 'Los Angeles', 'Lagos', 'Paris', 'Moscow']

for city in cities:
    model_summary = []
    for i in range(len(model_names)):
        if not os.path.exists(f'data/models/{city}/{model_names[i]}'):
            os.makedirs(f'data/models/{city}/{model_names[i]}')

        df = model_calls[i](city)
        m, forecast, df_cv = forecast_regressors(df, 6, 12)
        with open(f'data/models/{city}/{model_names[i]}/m.json', 'w') as fout:
                json.dump(model_to_json(m), fout)
        forecast.to_csv(f'data/models/{city}/{model_names[i]}/forecast.csv', index=False)
        cv_metrics = performance_metrics(df_cv, rolling_window=1)
        cv_metrics['Model'] = model_names[i]
        model_summary.append(cv_metrics)

        df.dropna(axis=0, inplace=True)
        df.to_csv(f'data/models/{city}/{model_names[i]}/{model_names[i]}_data.csv', index=False)
        summary_stats = get_summary_statistics(df)
        summary_stats.to_csv(f'data/models/{city}/{model_names[i]}/{model_names[i]}_stats.csv')
        clear_output()

    model_summary = pd.concat(model_summary)
    model_summary.to_csv(f'data/models/{city}/Summary_table.csv', index=False)